In [ ]:
# packages
import csv
from concurrent.futures import thread
import pandas as pd
import json
import pickle

In [ ]:
df_marieke = pd.read_csv('marieke_s.csv')
df_friso = pd.read_csv('friso_s.csv')
df_robin = pd.read_csv('robin_s.csv')

df_merged = pd.concat([df_marieke['thread_id'],
                       df_marieke['comment_id'],
                       df_marieke['sentence'],
                       df_marieke['evidence'],
                       df_friso['evidence'],
                       df_robin['evidence']],
                       axis=1)
df_merged['gold_label'] = ""
df_merged.columns = ['thread_id', 'comment_id', 'sentence', 'a1', 'a2', 'a3', 'gold_label']

for index, row in df_merged.iterrows():
    gold_label = ""
    a1 = row['a1']
    a2 = row['a2']
    a3 = row['a3']
    if a1 == 'Common ground':
        a1 = 'Assumption'
    if a2 == 'Common ground':
        a2 = 'Assumption'
    if a3 == 'Common ground':
        a3 = 'Assumption'
    if a1 == a2 or a1 == a3:
        gold_label = a1
    elif a2 == a3:
        gold_label = a2
    df_merged.at[index, 'gold_label'] = gold_label

df_merged.to_csv('10threads.csv', index=False)

# Remaining topics

In [ ]:
# code for remaining topics:
# three final files
files = ["final_friso.csv", "final_marieke.csv", "final_robin.csv"]
# list for labels
dataset = []
# list for the first 10 topics to filter them out
df = pd.read_csv('marieke.csv')
thread_ids = list(set(df['thread_id'].tolist()))
x = -1
row = len(df.index)
for fl in files:
    # iterate through the three files
    with open(fl) as f:
        # reads file
        csv_reader = csv.reader(f, delimiter=',')
        # because friso's csv file is generated differently we need to
        # set the index manually
        for line in csv_reader:
            # If line is the headers of a new file the line got skipped
            # Or if the first 10 topics are in the file
            if "final_friso" in fl:
                i = 10
            else:
                i = 5
            if line[0] == "thread_id" or line[0] in thread_ids or line[i] == "Double":
                continue

            # if there are somehow 2 labels for 1 sentence line is skipped
            # (only for 4 sentences this is the case)
            if not '{"choices":' in line[i]:
                if not line[i] == "":
                  df_merged.loc[row] = [line[0], line[1], line[2], "", "", "", line[i]]
                  row +=1




# Double Sentences

In [ ]:
# appending the gold_label of the total disagreement
# making a dict so we can add thread id and comment id as well
dict = {}
with open("marieke.csv") as f:
  csv_reader = csv.reader(f, delimiter=",")
  for line in csv_reader:
    if not line[2] == "sentence":
      dict[line[2]] = (line[0], line[1])




with open("disagreement_fixed.csv") as f:
        csv_reader = csv.reader(f, delimiter=',')
        for line in csv_reader:
          if not line[0] == "sentence":
            thread_id = dict[line[0]][0]
            comment_id = dict[line[0]][1]
            df_merged.loc[df_merged["sentence"] == line[0]] = [thread_id, comment_id, line[0], line[1], line[2], line[3], line[4]]
            row +=1


# Total disagreement labels

In [ ]:
with open("double_sentence_fixed.csv") as f:
        csv_reader = csv.reader(f, delimiter=',')
        for line in csv_reader:
            df_merged.loc[row] = [line[2], line[3], line[0], "", "", "", line[1]]
            row +=1
df_merged.to_csv('full_annotations.csv', index=False)
y = df_merged["gold_label"].tolist()
df = df_merged.drop(df_merged[df_merged.gold_label == ""].index)
# for index, row in df_merged.iterrows():
#   if row["gold_label"] == "Continue":
#     df_merged.at[index, 'gold_label'] = df_merged.at[index-1, "gold_label"]

df_merged.to_csv('dataset.csv', index=False)